In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder


#reads in the data
data = pd.read_csv('movie_metadata.csv')

#removes the values that aren't numeric
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'content_rating']

#makes the new data set without the to_drop colums
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 
   
print('Done')

Done


# Changing Genres into individual Columns

In [2]:
genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

set(['Sci-Fi', 'Crime', 'Romance', 'Animation', 'Music', 'Comedy', 'War', 'Horror', 'Film-Noir', 'Adventure', 'News', 'Reality-TV', 'Thriller', 'Western', 'Mystery', 'Short', 'Drama', 'Action', 'Documentary', 'Musical', 'History', 'Family', 'Fantasy', 'Game-Show', 'Sport', 'Biography'])


In [3]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

del movie_data['genres']
#print(genres_dict['Short'])

for genre in genres_set:
    series = pd.Series(genres_dict[genre])
    movie_data[genre] = series

#print(movie_data['Short'].values)    
print('Done')

Done


# Deleting NaN's

In [4]:
NA_THRESH = 4

#deletes the row when there are more or equal to the threshod number of NaN's
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data

#returns length
movie_data = remove_too_many_NaN(movie_data, NA_THRESH)

Length before deletion: 5043
Length after deletion: 4927


In [5]:
def replace_NaNs(col):
    # compute average
    avg = np.sum(col) / (len(col) - np.sum(col.isnull()))
    print(avg)
    
    # replace NaNs with average
    col = col.fillna(value=avg)
    return col

pd.options.mode.chained_assignment= None

to_replace_NaNs = ['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 
               'aspect_ratio', 'budget', 'cast_total_facebook_likes', 'director_facebook_likes', 'gross', 'title_year'
                   , 'duration', 'facenumber_in_poster']
for column in to_replace_NaNs:
    movie_data[column] = replace_NaNs(movie_data[column])


6615.96954933
1676.81555962
651.352522376
2.13102265372
39838985.8163
9802
689.276128508
48503328.2327
2002.44376652
108.152580252
1.36561546287


# Changing Color to Numeric

In [6]:
#replaces the color value to 1
movie_data.color = movie_data.color.replace(to_replace = 'Color', value = 1)
movie_data.color = movie_data.color.replace(to_replace = 'NaN', value = 1)

#replaces the NaN or black and white values to 0
for item in movie_data.color:
    if item != 1:
        movie_data.color = movie_data.color.replace(to_replace = item, value = 0)
        
#movie_data.color = new_color_column 
#print(movie_data['color'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Changing Country to Numeric

In [7]:
#replaces the USA values to 1
movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)
movie_data.country = movie_data.country.replace(to_replace ='NaN', value = 1)
#replaces the NaN or non USA values to 0
for item in movie_data.country:
    if item != 1:
        movie_data.country = movie_data.country.replace(to_replace = item, value = 0)
       
#print(movie_data['country'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None

print('Done')

Done


# Changing Languange to Numeric

In [8]:
#replaces the English values to 1
movie_data.language = movie_data.language.replace(to_replace = 'English', value = 1)
movie_data.language = movie_data.language.replace(to_replace = 'NaN', value = 1)

#replaces the other values to 0
for item in movie_data.language:
    if item != 1:
        movie_data.language = movie_data.language.replace(to_replace = item, value = 0)
        
#print(movie_data['language'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Chaning Content_Rating to Numeric

In [9]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

movie_data = content_to_numerical(movie_data)
print(movie_data)

      actor_1_facebook_likes  actor_2_facebook_likes  actor_3_facebook_likes  \
0                     1000.0                   936.0                   855.0   
1                    40000.0                  5000.0                  1000.0   
2                    11000.0                   393.0                   161.0   
3                    27000.0                 23000.0                 23000.0   
5                      640.0                   632.0                   530.0   
6                    24000.0                 11000.0                  4000.0   
7                      799.0                   553.0                   284.0   
8                    26000.0                 21000.0                 19000.0   
9                    25000.0                 11000.0                 10000.0   
10                   15000.0                  4000.0                  2000.0   
11                   18000.0                 10000.0                   903.0   
12                     451.0            

# Trainset en testset

In [10]:
ratings = movie_data['imdb_score'].values
del movie_data['imdb_score']

X = movie_data.values
X_std = StandardScaler().fit_transform(X)

number_of_samples = len(ratings)
np.random.seed(0)
random_indices = np.random.permutation(number_of_samples)
num_training_samples = int(number_of_samples*0.75)
x_train = X_std[random_indices[:num_training_samples]]
y_train = ratings[random_indices[:num_training_samples]]


x_test = X_std[random_indices[num_training_samples:]]
y_test = ratings[random_indices[num_training_samples:]]
y_train = list(y_train)


# Analysis prediction function

In [11]:
def analyse_predictions(regr, x_test, y_test, doprint=False):
    predres = list(regr.predict(x_test))
    actual = list(y_test)

    highest = lowest = avg = 0
    for i in range(len(predres)):
        diff = abs(predres[i] - actual[i])
        avg += diff

        if doprint:
            print "predicted:", predres[i], 
            print "\tActual:", actual[i],
            print "\tDifference:", abs(predres[i] - actual[i])

        if diff > highest:
            highest = diff
        if diff < lowest:
            lowest = diff

    print
    print "Highest difference:", highest
    print "Lowest difference:", lowest
    print "average difference:", avg / len(predres)

# Linear Regression

In [12]:
from sklearn import linear_model
regr = linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

regr.fit (x_train, y_train)
#
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

analyse_predictions(regr, x_test, y_test)


('Coefficients: \n', array([  8.22330619e-01,   2.70704535e-01,   7.27031670e-02,
        -1.58790889e-02,  -4.15781211e-02,  -9.46689391e-01,
        -2.98279713e-02,  -8.59713409e-02,   6.95311541e-02,
         1.58074141e-01,  -5.94081224e-02,   2.04331554e-01,
        -1.08128148e-01,   2.56613371e-01,  -2.86185735e-01,
        -2.74147959e-02,   5.99095136e-02,  -2.52863942e-02,
         1.44836221e-01,  -2.85213460e-02,  -5.40626026e-02,
        -7.27571207e-03,  -1.03066696e-01,  -9.23323020e-03,
        -3.33154229e-02,   2.08073427e-03,   1.00613962e-16,
        -7.40660887e-02,  -2.33183163e-02,   2.44412442e-02,
         2.87542092e-02,   2.34201612e-01,  -8.99457842e-02,
         1.67765646e-01,  -3.67860173e-02,  -1.28065037e-02,
        -6.22834473e-02,  -4.20552650e-02,   1.04083409e-17,
         1.91744173e-04,   4.47557634e-02]))
Mean squared error: 0.87
Variance score: 0.31

Highest difference: 6.31631416968
Lowest difference: 0
average difference: 0.688728841196


# Ridge Regression

In [13]:
regr = linear_model.Ridge()

regr.fit (x_train, y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

analyse_predictions(regr, x_test, y_test)


('Coefficients: \n', array([  7.08481783e-01,   2.38515318e-01,   5.30536847e-02,
        -1.57916653e-02,  -4.17431743e-02,  -8.07695056e-01,
        -2.98629903e-02,  -8.66312167e-02,   6.96390614e-02,
         1.57657267e-01,  -5.94827050e-02,   2.02540679e-01,
        -1.08449158e-01,   2.56625245e-01,  -2.86483283e-01,
        -2.73642000e-02,   5.97459347e-02,  -2.50957378e-02,
         1.44864941e-01,  -2.85760794e-02,  -5.45911173e-02,
        -7.04177959e-03,  -1.03117579e-01,  -9.18787419e-03,
        -3.27578086e-02,   2.07685273e-03,   0.00000000e+00,
        -7.40924447e-02,  -2.33951359e-02,   2.46108576e-02,
         2.88365704e-02,   2.34490123e-01,  -9.03353062e-02,
         1.68226964e-01,  -3.66472241e-02,  -1.28341281e-02,
        -6.22796991e-02,  -4.18769249e-02,   0.00000000e+00,
         2.39248358e-04,   4.46595679e-02]))
Mean squared error: 0.87
Variance score: 0.31

Highest difference: 6.3136604861
Lowest difference: 0
average difference: 0.688796063608


# Lasso

In [14]:
regr = linear_model.Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

regr.fit (x_train, y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

analyse_predictions(regr, x_test, y_test)

('Coefficients: \n', array([  0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,  -8.65889302e-05,   2.01728581e-02,
         1.35440908e-01,  -0.00000000e+00,   5.42200783e-02,
        -4.92742094e-02,   1.69494542e-01,  -1.59573130e-01,
        -0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -3.46402261e-02,   0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   1.95709349e-01,  -0.00000000e+00,
         7.74590406e-02,  -0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   3.31317105e-03]))
Mean squared error: 0.95
Variance score: 0.26

Highest difference: 5.55481229422
Lowest difference: 0
average difference: 0.728641679958


# Bayesian Ridge

In [15]:
regr = linear_model.BayesianRidge(alpha_1=1e-06, alpha_2=5e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

regr.fit (x_train, y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

analyse_predictions(regr, x_test, y_test)

('Coefficients: \n', array([  8.73526649e-02,   6.08666982e-02,  -5.07100466e-02,
        -1.56547004e-02,  -3.65857007e-02,  -5.00051417e-02,
        -3.16301475e-02,  -8.84259132e-02,   7.05010335e-02,
         1.54717559e-01,  -5.99621592e-02,   1.87944061e-01,
        -1.08417802e-01,   2.51021710e-01,  -2.80047933e-01,
        -2.67481869e-02,   5.80274298e-02,  -2.28595869e-02,
         1.39854761e-01,  -2.82000502e-02,  -5.83970877e-02,
        -4.32077712e-03,  -1.03197376e-01,  -7.75522162e-03,
        -2.84826624e-02,   2.60054476e-03,   1.36333265e-16,
        -7.30259136e-02,  -2.23763763e-02,   2.45793708e-02,
         2.85559477e-02,   2.30796933e-01,  -9.19987611e-02,
         1.66397400e-01,  -3.44052259e-02,  -1.20620081e-02,
        -5.96664661e-02,  -4.04025585e-02,   1.99053091e-16,
         7.04830660e-04,   4.44688042e-02]))
Mean squared error: 0.87
Variance score: 0.31

Highest difference: 6.26524939558
Lowest difference: 0
average difference: 0.68899764208
